# Relationship Extraction

## Student Info

In [73]:
# load data
student_info = pd.read_csv('../entity_extraction/student_info_final.csv') 
print(student_info.head())

    Student_Name Matric_Number       NRIC  Year                  Faculties  \
0    Tracy Lewis     A0216920B  XXXXX506Z     1        NUS Business School   
1    Tracy Lewis     A0216920B  XXXXX506Z     1        NUS Business School   
2    Tracy Lewis     A0216920B  XXXXX506Z     1        NUS Business School   
3  Andrew Holden     A0225069H  XXXXX799Z     3  YST Conservatory of Music   
4  Andrew Holden     A0225069H  XXXXX799Z     3  YST Conservatory of Music   

                     Major Second Major  \
0  Business Administration          NaN   
1  Business Administration          NaN   
2  Business Administration          NaN   
3                    Music          NaN   
4                    Music          NaN   

                                   Modules_Completed  \
0  ['ACC1701B', 'DMB1202ACC', 'DMB1201MKT', 'MNO1...   
1  ['ACC1701B', 'DMB1202ACC', 'DMB1201MKT', 'MNO1...   
2  ['ACC1701B', 'DMB1202ACC', 'DMB1201MKT', 'MNO1...   
3  ['CFA1111A', 'MUA1190', 'MUA2109', 'MUA1172',

In [75]:
# for each row, create a dictionary for each relationship and add to list

for i in range(len(student_info)):
    # student --> majors_in --> major
    student_major = {'from_type': 'Student', 'from_id': student_info['Student_Name'][i], 'to_type': 'Major', 'to_id': student_info['Major'][i], 'type': 'majors_in'}
    # student --> is_in_faculty --> faculty
    student_faculty = {'from_type': 'Student', 'from_id': student_info['Student_Name'][i], 'to_type': 'Faculty', 'to_id': student_info['Faculties'][i], 'type': 'is_in_faculty'}
    # student --> takes_module --> module
    student_module = {'from_type': 'Student', 'from_id': student_info['Student_Name'][i], 'to_type': 'Module', 'to_id': student_info['Modules_Completed'][i], 'type': 'takes_module'}
    
    student_info['Relationship_Data'][i].append(student_major)
    student_info['Relationship_Data'][i].append(student_faculty)
    student_info['Relationship_Data'][i].append(student_module)

# explode the list of dictionaries into separate rows
student_info = student_info.explode('Relationship_Data')

print(student_info)

        Student_Name Matric_Number       NRIC  Year            Faculties  \
0        Tracy Lewis     A0216920B  XXXXX506Z     1  NUS Business School   
0        Tracy Lewis     A0216920B  XXXXX506Z     1  NUS Business School   
0        Tracy Lewis     A0216920B  XXXXX506Z     1  NUS Business School   
1        Tracy Lewis     A0216920B  XXXXX506Z     1  NUS Business School   
1        Tracy Lewis     A0216920B  XXXXX506Z     1  NUS Business School   
...              ...           ...        ...   ...                  ...   
9853  Angel Peterson     A0218295X  XXXXX501Z     3  NUS Business School   
9853  Angel Peterson     A0218295X  XXXXX501Z     3  NUS Business School   
9854  Angel Peterson     A0218295X  XXXXX501Z     3  NUS Business School   
9854  Angel Peterson     A0218295X  XXXXX501Z     3  NUS Business School   
9854  Angel Peterson     A0218295X  XXXXX501Z     3  NUS Business School   

                        Major Second Major  \
0     Business Administration          Na

In [76]:
# write back to csv
student_info.to_csv('../entity_extraction/student_info_final.csv', index=False)

In [78]:
print(student_major)

In [6]:
print(student_faculty)

              Subject      Predicate                             Object
0         Tracy Lewis  is_in_faculty                NUS Business School
1       Andrew Holden  is_in_faculty          YST Conservatory of Music
2     Phillip Bullock  is_in_faculty  College of Design and Engineering
3      Valerie Rivera  is_in_faculty                          Computing
4         Robert Hall  is_in_faculty          YST Conservatory of Music
...               ...            ...                                ...
3280  Renee Carpenter  is_in_faculty                NUS Business School
3281    Garrett Mcgee  is_in_faculty                                Law
3282    Michael Colon  is_in_faculty       Yong Loo Lin Sch of Medicine
3283     Wesley Moran  is_in_faculty                                Law
3284   Angel Peterson  is_in_faculty                NUS Business School

[3285 rows x 3 columns]


In [7]:
# create new df
student_module = pd.DataFrame()

# student --> takes_module --> module 
# for every module in 'Module', create a new row in the df

student_module['Subject'] = student_info['Student_Name']
student_module['Predicate'] = 'takes_module'
student_module['Object'] = student_info['Modules_Completed']
# Convert comma-separated strings to lists
student_module['Object'] = student_module['Object'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
student_module = student_module.explode('Object')
student_module['Predicate2'] = 'and_scored'
student_module['Object2'] = student_info['Grades']
# Convert string representations of dictionaries into actual dictionaries
student_module['Object2'] = student_module['Object2'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# grades column should be the value of key in 'Object' only
student_module['Object2'] = student_module.apply(
    lambda row: row['Object2'].get(row['Object'], None), axis=1
)

print(student_module)


             Subject     Predicate      Object  Predicate2 Object2
0        Tracy Lewis  takes_module    ACC1701B  and_scored       B
0        Tracy Lewis  takes_module  DMB1202ACC  and_scored       A
0        Tracy Lewis  takes_module  DMB1201MKT  and_scored      C+
0        Tracy Lewis  takes_module    MNO1706B  and_scored       C
0        Tracy Lewis  takes_module      RE1702  and_scored      C+
...              ...           ...         ...         ...     ...
3284  Angel Peterson  takes_module    ACC1701C  and_scored       D
3284  Angel Peterson  takes_module     BSE3711  and_scored      B+
3284  Angel Peterson  takes_module  DMB1204ACC  and_scored       D
3284  Angel Peterson  takes_module      RE3000  and_scored       D
3284  Angel Peterson  takes_module     BI3704B  and_scored      B-

[65104 rows x 5 columns]


In [8]:
# load data
module_info = pd.read_csv('../entity_extraction/module_info_final.csv')

In [9]:
# create new df
module_department = pd.DataFrame()

# module --> offered_by --> department
module_department['Subject'] = module_info['moduleCode']
module_department['Predicate'] = 'offered_by'
module_department['Object'] = module_info['department']

# remove nan values
module_department = module_department.dropna()

print(module_department)

      Subject   Predicate                      Object
0     ABM5001  offered_by  NUS Medicine Dean's Office
1     ABM5002  offered_by  NUS Medicine Dean's Office
2     ABM5003  offered_by  NUS Medicine Dean's Office
3     ABM5004  offered_by  NUS Medicine Dean's Office
4     ABM5101  offered_by  NUS Medicine Dean's Office
...       ...         ...                         ...
6844   ZB3311  offered_by         Biological Sciences
6845   ZB3312  offered_by         Biological Sciences
6846   ZB4171  offered_by         Biological Sciences
6847   ZB4199  offered_by         Biological Sciences
6848   ZB4299  offered_by         Biological Sciences

[6849 rows x 3 columns]


In [11]:
# create new df
module_preclusion = pd.DataFrame()

# module --> has_preclusion --> preclusion
module_preclusion['Subject'] = module_info['moduleCode']
module_preclusion['Predicate'] = 'has_preclusion'
module_preclusion['Object'] = module_info['preclusion']

# keep only those with preclusions, not null and not empty
module_preclusion = module_preclusion[module_preclusion['Object'].notnull()]
module_preclusion = module_preclusion[module_preclusion['Object'] != '[]']

# explode the list of preclusions
module_preclusion['Object'] = module_preclusion['Object'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
module_preclusion = module_preclusion.explode('Object')

print(module_preclusion)



       Subject       Predicate    Object
20    ACC1701A  has_preclusion   ACC1002
20    ACC1701A  has_preclusion  ACC1002X
20    ACC1701A  has_preclusion   ACC1701
20    ACC1701A  has_preclusion    EC2204
20    ACC1701A  has_preclusion    RE1705
...        ...             ...       ...
6845    ZB3312  has_preclusion   PHS3312
6845    ZB3312  has_preclusion    QF3312
6845    ZB3312  has_preclusion    ST3312
6846    ZB4171  has_preclusion  YSC4211C
6848    ZB4299  has_preclusion    ZB4199

[8138 rows x 3 columns]


In [12]:
# create new df
module_skill = pd.DataFrame()

# module --> teaches_skill --> skill
module_skill['Subject'] = module_info['moduleCode']
module_skill['Predicate'] = 'teaches_skill'

# select values of key 'skills' from 'Description_entities' column
module_skill['Object'] = module_info['Description_entities'].apply(lambda x: ast.literal_eval(x).get('Skill', None) if isinstance(x, str) else x)    

# explode the list of skills
module_skill = module_skill.explode('Object')
#module_skill['Object'] = module_skill['Object'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# keep only those with skills, remove null and not empty
module_skill = module_skill[module_skill['Object'].notnull()]
module_skill = module_skill[module_skill['Object'] != '[]']

print(module_skill)

      Subject      Predicate       Object
0     ABM5001  teaches_skill   Leadership
1     ABM5002  teaches_skill     Research
1     ABM5002  teaches_skill      Extract
1     ABM5002  teaches_skill   Healthcare
2     ABM5003  teaches_skill     Research
...       ...            ...          ...
6843   ZB3310  teaches_skill  Environment
6844   ZB3311  teaches_skill  Environment
6845   ZB3312  teaches_skill  Environment
6846   ZB4171  teaches_skill     Research
6846   ZB4171  teaches_skill      Biology

[14947 rows x 3 columns]


## Department Info

In [14]:
# load data
department_info = pd.read_csv('../entity_extraction/department_info_final.csv')

## Staff Info

In [16]:
# load data
staff_info = pd.read_csv('../entity_extraction/staff_info_final.csv')

In [18]:
# create new df
staff_department = pd.DataFrame()

# staff --> is_in_department --> department
staff_department['Subject'] = staff_info['Employee Name']
staff_department['Predicate'] = 'is_in_department'
staff_department['Object'] = staff_info['Department_ID']

print(staff_department)

                    Subject         Predicate  \
0    Marin Sergio Hernandez  is_in_department   
1           Kathryn Cordova  is_in_department   
2           Barbara Sanchez  is_in_department   
3               Bryce Lucas  is_in_department   
4            Judith Camacho  is_in_department   
..                      ...               ...   
995              Lisa Reyes  is_in_department   
996           Melanie Kelly  is_in_department   
997          Ronald Spencer  is_in_department   
998          Charles Rogers  is_in_department   
999          Vanessa Taylor  is_in_department   

                                  Object  
0    Electrical and Computer Engineering  
1    Civil and Environmental Engineering  
2            Centre for Language Studies  
3                      BIZ Dean's Office  
4                              Economics  
..                                   ...  
995                  YSTCM Dean's Office  
996                    FoL Dean's Office  
997                     

In [ ]:
# write all back to original csv files
student_major.to_csv('../entity_extraction/student_major.csv', index=False)
student_faculty.to_csv('../entity_extraction/student_faculty.csv', index=False)
student_module.to_csv('../entity_extraction/student_module.csv', index=False)

In [181]:
# write all back to  csv files


student_major.to_csv('student_major.csv', index=False)
student_faculty.to_csv('student_faculty.csv', index=False)
student_module.to_csv('student_module.csv', index=False)
module_department.to_csv('module_department.csv', index=False)
module_prerequisite.to_csv('module_prerequisite.csv', index=False)
module_preclusion.to_csv('module_preclusion.csv', index=False)
module_skill.to_csv('module_skill.csv', index=False)
module_semester.to_csv('module_semester.csv', index=False)
department_faculty.to_csv('department_faculty.csv', index=False)
staff_module.to_csv('staff_module.csv', index=False)
staff_department.to_csv('staff_department.csv', index=False)
